In [73]:
import glob
import pathlib
import pretty_midi
import fluidsynth

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from IPython import display
from typing import defaultdict

In [26]:
seed = 42
np.random.seed(42)
tf.random.set_seed(42)
SAMPLING_RATE = 20000

In [27]:
data_dir = pathlib.Path('data/maestro-v2.0.0/')
if not data_dir.exists():
    tf.keras.utils.get_file(
        'maestro-v2.0.0',
        origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
        extract = True,
        cache_dir='.',
        cache_subdir='data'
    )

In [28]:
filenames = glob.glob(str(data_dir/'**/*.mid*'))

In [34]:
sample = filenames[567]

In [35]:
sample

'data/maestro-v2.0.0/2006/MIDI-Unprocessed_06_R1_2006_01-04_ORIG_MID--AUDIO_06_R1_2006_01_Track01_wav.midi'

In [36]:
pm = pretty_midi.PrettyMIDI(sample)
pm

In [37]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds = 15):
    waveform = pm.fluidsynth(fs=SAMPLING_RATE)
    waveform_short = waveform[:seconds * SAMPLING_RATE]
    return display.Audio(waveform_short, rate=SAMPLING_RATE)

In [38]:
display_audio(pm)

In [40]:
instrument = pm.instruments[0]

In [44]:
pretty_midi.program_to_instrument_name(instrument.program)

'Acoustic Grand Piano'

In [50]:
instrument = pm.instruments[0]

In [51]:
instrument.program

0

In [52]:
pretty_midi.program_to_instrument_name(0)

'Acoustic Grand Piano'

# Extract the notes

In [69]:
len(filenames)

1282

In [75]:
num_files = 10
all_notes = []
for file in filenames[:10]:
    pm = pretty_midi.PrettyMIDI(file)
    instrument = pm.instruments[0]
    notes = defaultdict(list)
    
    sorted_notes = sorted(instrument.notes, key = lambda note: note.start)
    previous_start = sorted_notes[0].start
    
    for note in sorted_notes:
        print(note)
        break

Note(start=0.994792, end=1.087240, pitch=77, velocity=56)
Note(start=1.001302, end=1.079427, pitch=63, velocity=49)
Note(start=1.028646, end=1.132812, pitch=77, velocity=66)
Note(start=1.006510, end=1.032552, pitch=68, velocity=54)
Note(start=1.007812, end=2.014323, pitch=67, velocity=71)
Note(start=0.787760, end=0.842448, pitch=76, velocity=79)
Note(start=0.992188, end=1.967448, pitch=67, velocity=74)
Note(start=0.997396, end=1.032552, pitch=56, velocity=74)
Note(start=0.701823, end=1.132812, pitch=63, velocity=99)
Note(start=0.838542, end=0.889323, pitch=36, velocity=63)


In [79]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
    pm = pretty_midi.PrettyMIDI(midi_file)
    instrument = pm.instruments[0]
    notes = defaultdict(list)
    
    # sort the notes by starting time
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    previous_start = sorted_notes[0].start
    
    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - previous_start)
        notes['duration'].append(end - start)
        previous_start = start
        
    return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [81]:
df = midi_to_notes(sample)
df

,pitch,start,end,step,duration
0,65,0.957031,1.946615,0.000000,0.989583
1,53,0.977865,1.683594,0.020833,0.705729
2,57,1.269531,1.466146,0.291667,0.196615
3,60,1.457031,1.580729,0.187500,0.123698
4,57,1.623698,1.829427,0.166667,0.205729
...,...,...,...,...,...
6724,67,899.031250,900.200521,0.002604,1.169271
6725,58,899.036458,899.690104,0.005208,0.653646
6726,53,899.057292,901.962240,0.020833,2.904948
6727,57,901.053385,901.916667,1.996094,0.863281


In [88]:
all_notes = []
for file in filenames[:42]:
    df = midi_to_notes(file)
    all_notes.append(df)
    
all_notes = pd.concat(all_notes)

In [89]:
all_notes

,pitch,start,end,step,duration
0,77,0.994792,1.087240,0.000000,0.092448
1,49,0.998698,1.401042,0.003906,0.402344
2,73,1.108073,1.173177,0.109375,0.065104
3,68,1.207031,1.268229,0.098958,0.061198
4,73,1.315104,1.375000,0.108073,0.059896
...,...,...,...,...,...
6141,60,455.872396,456.065104,0.001302,0.192708
6142,44,455.872396,456.066406,0.000000,0.194010
6143,63,455.877604,456.100260,0.005208,0.222656
6144,32,455.894531,456.029948,0.016927,0.135417


In [ ]:
sequence_length = sequence_length + 1
windows = dataset